<a href="https://colab.research.google.com/github/eduseiti/ia368v_dd_class_10/blob/main/multi_stage_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys

import tqdm

In [2]:
IN_COLAB='google.colab' in sys.modules
LINK_WITH_COMET=False

In [3]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/aula_10"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)

    # Setup Anserini executables

    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
    os.environ["ANSERINI_CLASSPATH"]="/content/drive/MyDrive/unicamp/ia368v_dd/anserini/target"

    !chmod +x /content/drive/MyDrive/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval
    

    !pip install pyserini -q
    !pip install faiss-cpu -q

    PYSERINI_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/pyserini/"

    TREC_EVAL_FULLPATH=PYSERINI_FOLDER+"tools/eval/trec_eval.9.0.4/trec_eval"
    
    PYTHON="python3"
    
    !pip install transformers -q

else:
    PYTHON="python3.8"
    
    WORKING_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/ia368v_dd_class_10/"
    PYSERINI_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/pyserini/"
    
    TREC_EVAL_FULLPATH=PYSERINI_FOLDER+"tools/eval/trec_eval.9.0.4/trec_eval"
    
    os.environ["ANSERINI_CLASSPATH"]="/media/eduseiti/bigdata01/unicamp/ia368v_dd/anserini/target"
    
PYSERINI_TOOLS_FOLDER=PYSERINI_FOLDER+"tools/scripts/msmarco/"

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 108.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 78.6 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np

import os
import pickle

import time

from datetime import datetime

from scipy import stats

import sys

In [5]:
from pyserini.search.lucene import LuceneSearcher

In [6]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, BatchEncoding

In [7]:
RETURN_OVERFLOWING_TOKENS=True
USE_DOC2QUERY_EXPANDED_CORPUS=False
USE_DOC2QUERY_EXPANDED_CORPUS_ONLY_FOR_BM25=True

USE_DPR_SECOND_SEARCHER=True
DPR_EXHAUSTIVE_SEARCH=True

In [8]:
if RETURN_OVERFLOWING_TOKENS:
    RERANKING_TOKENIZED_DOCS_FILE_FORMAT="{}_tokenized_overflow_reranking_{}.pkl"
else:
    RERANKING_TOKENIZED_DOCS_FILE_FORMAT="{}_tokenized_reranking_{}.pkl"

MAX_TOKENS_LENGTH=512

MODEL_NAME="cross-encoder/ms-marco-MiniLM-L-6-v2"
MODEL_CHECKPOINT="trained_models/checkpoint_eduseiti_1000_queries_expansion_20230502_03.jsonl_03_epoch_20230508_005045_0.0719"

DPR_MODEL_NAME="microsoft/MiniLM-L12-H384-uncased"
DPR_MODEL_CHECKPOINT="trained_single_model/checkpoint_epoch_5_20230418_170302_0.0689"

In [9]:
TREC_COVID_ORIGINAL_TITLE_TEXT_MERGED_FILENAME="trec_covid_original_title_text_merged.tsv"
TREC_COVID_EXPANDED_TITLE_TEXT_MERGED_FILENAME="trec_covid_expanded.tsv"

TREC_COVID_QUERIES="trec_covid_queries.tsv"
TREC_COVID_QRELS="trec_covid_qrels.tsv"

TREC_COVID_ORIGINAL_FOLDER="trec_covid_original"
TREC_COVID_ORIGINAL_INDEX_FOLDER="trec_covid_original/index"
TREC_COVID_ORIGINAL_RUNS_FOLDER="trec_covid_original/runs"

TREC_COVID_EXPANDED_FOLDER="trec_covid_expanded"
TREC_COVID_EXPANDED_INDEX_FOLDER="trec_covid_expanded/index"
TREC_COVID_EXPANDED_RUNS_FOLDER="trec_covid_expanded/runs"

TREC_COVID_RERANKING_RUNS_FOLDER="trec_covid_reranking_runs"

TREC_COVID_DPR_ENCODED_FILENAME="trec_covid_DPR_encoded_{}.pkl"
TREC_COVID_DPR_INDEX_FILENAME="trec_covid_DPR_index_{}.pkl"

if USE_DOC2QUERY_EXPANDED_CORPUS:
    PYSERINI_TEST_RUN_RERANKED_FILENAME_FORMAT="run.trec_covid_expanded_reranking_{}_{}.txt"
    RESULTS_FILENAME="expanded_bm25_inpars_reranking_t4_15GB.pkl"
elif USE_DOC2QUERY_EXPANDED_CORPUS_ONLY_FOR_BM25:
    PYSERINI_TEST_RUN_RERANKED_FILENAME_FORMAT="run.trec_covid_expanded_BM25_only_reranking_{}_{}.txt"
    RESULTS_FILENAME="expanded_bm25_only_inpars_reranking_t4_15GB.pkl"
else:
    PYSERINI_TEST_RUN_RERANKED_FILENAME_FORMAT="run.trec_covid_reranking_{}_{}.txt"
    RESULTS_FILENAME="original_bm25_inpars_reranking_t4_15GB.pkl"

# Initial setup

In [10]:
def create_BM25_index(corpus_original_file, converted_corpus_folder, index_folder):

    bm25_index_creation_start_time = time.time()

    if os.path.exists(index_folder):
        print("Index already created...")
    else:
        # Convert corpus to Pyserini's format

        !{PYTHON} {PYSERINI_TOOLS_FOLDER}/convert_collection_to_jsonl.py \
            --collection-path {corpus_original_file} \
            --output-folder {converted_corpus_folder}


        # Create the BM25 index

        current_folder = os.getcwd()

        os.chdir(PYSERINI_FOLDER)

        !{PYTHON} -m pyserini.index.lucene \
            --collection JsonCollection \
            --input {current_folder}/{converted_corpus_folder} \
            --index {current_folder}/{index_folder} \
            --generator DefaultLuceneDocumentGenerator \
            --threads 9 \
            --storePositions --storeDocvectors --storeRaw

        os.chdir(current_folder)

    return time.time() - bm25_index_creation_start_time

### Tokenization functions

In [11]:
def tokenize_documents_for_reranking(corpus_original_file, pandas_read_parameters, id_column, text_column, tokenizer, model_name):

    docs_tokenization_start_time = time.time()

    tokenized_documents_filename = RERANKING_TOKENIZED_DOCS_FILE_FORMAT.format(os.path.splitext(corpus_original_file)[0],
                                                                               os.path.basename(model_name))

    if os.path.exists(tokenized_documents_filename):

        with open(tokenized_documents_filename, "rb") as input_file:
            tokenized_documents_data = pickle.load(input_file)

        documents_tokens_df = tokenized_documents_data['documents_tokens_df']
        overflow_to_sample_mapping = tokenized_documents_data['overflow_to_sample_mapping']

        print("Loaded pre-tokenized documents...")
    else:
        print("Need to tokenize the corpus...")

        corpus_original_df = pd.read_csv(corpus_original_file, **pandas_read_parameters)

#         display(corpus_original_df)

#         print("id_column={}, text_column={}".format(id_column, text_column))

        corpus_original_ids_list = corpus_original_df[id_column].tolist()

        documents_tokens = tokenizer(corpus_original_df[text_column].tolist(), 
                                     truncation=True,
                                     return_overflowing_tokens=RETURN_OVERFLOWING_TOKENS, 
                                     max_length=MAX_TOKENS_LENGTH, 
                                     return_length=True)
        
        print(stats.describe(documents_tokens['length']))
        
        #### Check if has truncated documents

        original_length = len(corpus_original_ids_list)

        if 'overflow_to_sample_mapping' in documents_tokens:
            if original_length < len(documents_tokens['overflow_to_sample_mapping']):
                print("Added {} overflowing texts...".format(len(documents_tokens['overflow_to_sample_mapping']) - original_length))
        else:
            # Add the field to not break the code

            documents_tokens['overflow_to_sample_mapping'] = np.array(list(range(original_length)))


        documents_tokens_df = pd.DataFrame([corpus_original_ids_list[i] for i in documents_tokens['overflow_to_sample_mapping']], columns=["doc_id"])
        documents_tokens_df['input_ids'] = documents_tokens['input_ids']
        documents_tokens_df['token_id'] = list(range(documents_tokens_df.shape[0]))
        

        #### Save the tokenized data

        with open(tokenized_documents_filename, "wb") as output_file:
            pickle.dump({'documents_tokens_df': documents_tokens_df,
                         'overflow_to_sample_mapping': documents_tokens['overflow_to_sample_mapping']}, output_file, pickle.HIGHEST_PROTOCOL)
            
        overflow_to_sample_mapping = documents_tokens['overflow_to_sample_mapping']
            
    return documents_tokens_df, overflow_to_sample_mapping, time.time() - docs_tokenization_start_time

In [12]:
def tokenize_corpus_for_reranking(which_corpus_file, hf_tokenizer_reference, original_document_pandas_parameters):

    tokenizer = AutoTokenizer.from_pretrained(hf_tokenizer_reference)

    documents_tokens_df, overflow_to_sample_mapping, elapsed_time = tokenize_documents_for_reranking(which_corpus_file, 
                                                                                                     original_document_pandas_parameters, 
                                                                                                     original_document_pandas_parameters['names'][0], 
                                                                                                     original_document_pandas_parameters['names'][1], 
                                                                                                     tokenizer, 
                                                                                                     MODEL_NAME)

    print(elapsed_time)

    return tokenizer, documents_tokens_df, overflow_to_sample_mapping, elapsed_time

### DPR related functions

In [13]:
class TextToEncodeDataset(torch.utils.data.Dataset):

    def __init__(self, documents_tokens_df, overflow_to_sample_mapping):

        self.documents_tokens_df = documents_tokens_df
        self.overflow_to_sample_mapping = overflow_to_sample_mapping


    def __len__(self):
        return self.documents_tokens_df.shape[0]


    def __getitem__(self, index):
        return {'input_ids': self.documents_tokens_df.iloc[index]['input_ids'],
                'attention_mask': [1] * len(self.documents_tokens_df.iloc[index]['input_ids'])}

In [42]:
class QueryToEncodeDataset(torch.utils.data.Dataset):

    def __init__(self, query_tokens):

        self.query_tokens = query_tokens


    def __len__(self):
        return 1


    def __getitem__(self, index):
        return {'input_ids': self.query_tokens['input_ids'],
                'attention_mask': self.query_tokens['attention_mask']}

In [15]:
class DPRCollator(object):
    def __init__(self, tokenizer=None):
        self.tokenizer = tokenizer


    def __call__(self, batch):
        padded_batch = self.tokenizer.pad(batch, return_tensors='pt')

        return BatchEncoding(padded_batch)

In [16]:
def DPR_encode(which_model, 
               which_dataloader):
    
    encoded_text = []

    which_model.eval()

    with torch.no_grad():
        for batch in which_dataloader:
            encoded_outputs = which_model(**batch.to(device))

            data_shape = encoded_outputs.last_hidden_state.shape

            encoded_cls = encoded_outputs.last_hidden_state[:, 0, :].view(data_shape[0], -1)

            encoded_text.append(encoded_cls.cpu().numpy())

    return np.concatenate(encoded_text, axis=0)

In [17]:
def create_DPR_index_faiss(documents_tokens_df, overflow_to_sample_mapping):

    start_time = time.time()
    
    if os.path.exists(TREC_COVID_DPR_INDEX_FILENAME.format(os.path.basename(DPR_MODEL_NAME))):
        print("DPR index already computed...")
        
        with open(TREC_COVID_DPR_INDEX_FILENAME.format(os.path.basename(DPR_MODEL_NAME)), "rb") as input_file:
            corpus_index = pickle.load(input_file)
    else:
        if os.path.exists(TREC_COVID_DPR_ENCODED_FILENAME.format(os.path.basename(DPR_MODEL_NAME))):
            print("DPR encoded corpus alread computed...")
            
            with open(TREC_COVID_DPR_ENCODED_FILENAME.format(os.path.basename(DPR_MODEL_NAME)), "rb") as input_file:
                encoded_corpus = pickle.load(input_file)
        else:
            corpus_to_encode = TextToEncodeDataset(documents_tokens_df, overflow_to_sample_mapping)

            encode_corpus_dataloader = torch.utils.data.DataLoader(corpus_to_encode, 
                                                                   batch_size=DPR_batch_size, 
                                                                   shuffle=False, 
                                                                   collate_fn=DPRCollator(tokenizer))

            encoded_corpus = DPR_encode(DPR_model, encode_corpus_dataloader)
    
        corpus_index = faiss.IndexFlatIP(encoded_corpus.shape[-1])

        corpus_index.add(encoded_corpus)

    print("Time to create the DPR index: {}".format(time.time - start_time()))
        
    return corpus_index

In [18]:
def create_DPR_index(documents_tokens_df, overflow_to_sample_mapping):

    start_time = time.time()
    
    if os.path.exists(TREC_COVID_DPR_ENCODED_FILENAME.format(os.path.basename(DPR_MODEL_NAME))):
        print("DPR encoded corpus alread computed...")

        with open(TREC_COVID_DPR_ENCODED_FILENAME.format(os.path.basename(DPR_MODEL_NAME)), "rb") as input_file:
            encoded_corpus = pickle.load(input_file)
    else:
        corpus_to_encode = TextToEncodeDataset(documents_tokens_df, overflow_to_sample_mapping)

        encode_corpus_dataloader = torch.utils.data.DataLoader(corpus_to_encode, 
                                                               batch_size=DPR_batch_size, 
                                                               shuffle=False, 
                                                               collate_fn=DPRCollator(tokenizer))

        encoded_corpus = DPR_encode(DPR_model, encode_corpus_dataloader)
    
        with open(TREC_COVID_DPR_ENCODED_FILENAME.format(os.path.basename(DPR_MODEL_NAME)), "wb") as output_file:
            pickle.dump(encoded_corpus, output_file, pickle.HIGHEST_PROTOCOL)

    print("Time to create the DPR index: {}".format(time.time() - start_time))
        
    return encoded_corpus

### Define the corpus information

In [19]:
TREC_COVID_ORIGINAL_INFO={
    "corpus_original_file": TREC_COVID_ORIGINAL_TITLE_TEXT_MERGED_FILENAME,
    "converted_corpus_folder": TREC_COVID_ORIGINAL_FOLDER,
    "index_folder": TREC_COVID_ORIGINAL_INDEX_FOLDER
}

TREC_COVID_ORIGINAL_PANDAS_PARAMS={
    "sep": '\t', 
    "header": None, 
    "names": ['doc_id', 'doc_title_text']
}

TREC_COVID_EXPANDED_INFO={
    "corpus_original_file": TREC_COVID_EXPANDED_TITLE_TEXT_MERGED_FILENAME,
    "converted_corpus_folder": TREC_COVID_EXPANDED_FOLDER,
    "index_folder": TREC_COVID_EXPANDED_INDEX_FOLDER
}

TREC_COVID_EXPANDED_PANDAS_PARAMS={
    "sep": '\t', 
    "header": None, 
    "names": ['_id', 'title_text']
}

if USE_DOC2QUERY_EXPANDED_CORPUS:
    bm25_parameters = TREC_COVID_EXPANDED_INFO
    corpus_loading_parameters = TREC_COVID_EXPANDED_PANDAS_PARAMS

elif USE_DOC2QUERY_EXPANDED_CORPUS_ONLY_FOR_BM25:
    bm25_parameters = TREC_COVID_EXPANDED_INFO
    corpus_loading_parameters = TREC_COVID_EXPANDED_PANDAS_PARAMS

    reranking_parameters = TREC_COVID_ORIGINAL_INFO
    reranking_corpus_loading_parameters = TREC_COVID_ORIGINAL_PANDAS_PARAMS
else:
    bm25_parameters = TREC_COVID_ORIGINAL_INFO
    corpus_loading_parameters = TREC_COVID_ORIGINAL_PANDAS_PARAMS

### BM25 documents index creation

In [20]:
create_BM25_index(**bm25_parameters)

Index already created...


0.0008780956268310547

In [21]:
if USE_DOC2QUERY_EXPANDED_CORPUS_ONLY_FOR_BM25:
    create_BM25_index(**reranking_parameters)

Index already created...


### Documents tokenization for reranking

In [22]:
if USE_DOC2QUERY_EXPANDED_CORPUS_ONLY_FOR_BM25:
    tokenizer, documents_tokens_df, overflow_to_sample_mapping, elapsed_time = tokenize_corpus_for_reranking(reranking_parameters['corpus_original_file'], MODEL_NAME, reranking_corpus_loading_parameters)    
else:
    tokenizer, documents_tokens_df, overflow_to_sample_mapping, elapsed_time = tokenize_corpus_for_reranking(bm25_parameters['corpus_original_file'], MODEL_NAME, corpus_loading_parameters)

Loaded pre-tokenized documents...
4.105802059173584


### Some initializations for the DL models

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### DPR index creation

In [24]:
if USE_DPR_SECOND_SEARCHER:
    DPR_model = AutoModel.from_pretrained(DPR_MODEL_CHECKPOINT).to(device)
    DPR_batch_size = 1024
    DPR_corpus_index = create_DPR_index(documents_tokens_df, overflow_to_sample_mapping)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Time to create the DPR index: 548.3242025375366


### Load the reranking model and create the searcher

In [25]:
reranking_model = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT).to(device)

In [26]:
batch_size=1024

In [27]:
if USE_DOC2QUERY_EXPANDED_CORPUS or USE_DOC2QUERY_EXPANDED_CORPUS_ONLY_FOR_BM25:
    BM25_searcher = LuceneSearcher(TREC_COVID_EXPANDED_INDEX_FOLDER)
else:
    BM25_searcher = LuceneSearcher(TREC_COVID_ORIGINAL_INDEX_FOLDER)

BM25_searcher.set_bm25(k1=float(0.82), b=float(0.68))

# Query execution

In [28]:
def collate_fn(batch):
    r1 = tokenizer.pad(batch, return_tensors='pt')

    return BatchEncoding(r1)

In [29]:
class RerankingData(torch.utils.data.Dataset):
    def __init__(self, examples, targets):
        self.examples = examples
        self.targets = targets
    
    def __len__(self):
        return len(self.examples['input_ids'])
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.examples['input_ids'][idx],
            'attention_mask': self.examples['attention_mask'][idx],
            'token_type_ids': self.examples['token_type_ids'][idx],
            'labels': int(self.targets[idx]),
            # 'labels': float(self.targets[idx]),
        }

In [30]:
def collect_reranking(model, dataloader, set_name=None):
    scores = []
    
    model.eval()

    with torch.no_grad():
        for batch in dataloader:
        # for batch in tqdm.tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False, bar_format='{l_bar}{bar:20}{r_bar}{bar:-20b}', colour='GREEN', file=sys.stdout, position=0, leave=True):
            
            # print("batch['input_ids'].shape={}".format(batch['input_ids'].shape))

            outputs = model(**batch.to(device))

            # print("outputs.logits.shape={}".format(outputs.logits.shape))

            scores.extend(outputs.logits.cpu().view(-1).numpy())

    return scores

In [59]:
def execute_DPR_search(query_tokens, first_search_docs_tokens_df, consolidation_approach='mean', verbose=False):
    
    start_time = time.time()
    
    query_dataloader = torch.utils.data.DataLoader(QueryToEncodeDataset(query_tokens), 
                                                   batch_size=1, 
                                                   shuffle=False, 
                                                   collate_fn=DPRCollator(tokenizer))
    
    encoded_query = DPR_encode(DPR_model, query_dataloader)
        
    # all_scores = torch.mm(encoded_query, DPR_corpus_index.t()).cpu().numpy()
    all_scores = torch.mm(torch.from_numpy(encoded_query), torch.from_numpy(DPR_corpus_index).t()).squeeze()
    
    # Filter the documents returned in the first search
    
    first_search_docs_tokens_indexes = first_search_docs_tokens_df.index.tolist()
    
    all_scores = all_scores[first_search_docs_tokens_indexes]
    
    doc_scores = {}
    first_search_doc_ids = first_search_docs_tokens_df['doc_id'].tolist()

    for i, match_score in enumerate(all_scores):
        which_doc = first_search_doc_ids[i]

        if which_doc not in doc_scores:
            doc_scores[which_doc] = []

        doc_scores[which_doc].append(match_score)
        
    consolidated_scores = []

    for document, scores in doc_scores.items():
        if consolidation_approach == 'mean':
            consolidated_scores.append({'doc_id': document, 'score': np.mean(scores)})
        else:
            consolidated_scores.append({'doc_id': document, 'score': np.max(scores)})
            
    consolidated_scores_df = pd.DataFrame(consolidated_scores).sort_values('score', ascending=False)    
    
    if verbose:
        print("DPR search duration: {}".format(time.time() - start_time))

    return consolidated_scores_df

In [60]:
def execute_query(query, documents_tokens_df=documents_tokens_df, k=1000, k2=1000, batch_size=batch_size, consolidation_approach='mean', verbose=False):

    query_start_time = time.time()

    hits = BM25_searcher.search(query, k=k)

    hits_df = []

    for i, hit in enumerate(hits):
        hits_df.append({'doc_id': hit.docid,
                        'score': hit.score})

    hits_df = pd.DataFrame(hits_df)

    # tokenize the query
    
    query_tokens = tokenizer(query, 
                             truncation=True,
                             return_overflowing_tokens=False, 
                             max_length=MAX_TOKENS_LENGTH, 
                             return_length=True)
    
    
    if USE_DPR_SECOND_SEARCHER:
        hits_df = execute_DPR_search(query_tokens, 
                                     documents_tokens_df.merge(hits_df.iloc[:k2], how='inner'),
                                     consolidation_approach=consolidation_approach)
        
    reranking_docs_tokens_df = documents_tokens_df.merge(hits_df.iloc[:k2], how='inner')

    
    end_token = [query_tokens['input_ids'][-1]]

    test_input_ids = []
    test_token_type_ids = []
    test_attention_mask = []

    for doc_input_id in reranking_docs_tokens_df['input_ids'].tolist():
        doc_trunked_input_id = doc_input_id[1:(MAX_TOKENS_LENGTH - query_tokens['length'][0])][:-1] + end_token
        test_input_ids.append(query_tokens['input_ids'] + doc_trunked_input_id)
        test_token_type_ids.append(query_tokens['token_type_ids'] + [1] * len(doc_trunked_input_id))
        test_attention_mask.append(query_tokens['attention_mask'] + [1] * len(doc_trunked_input_id))

    x_test = {'input_ids': test_input_ids, 
              'token_type_ids': test_token_type_ids, 
              'attention_mask': test_attention_mask}

    y_test = np.ones(len(test_input_ids), dtype=bool)

    dataset_test = RerankingData(x_test, y_test)
    dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    # print("len(dataset_test)={}".format(len(dataset_test)))

    reranking_start_time = time.time()

    reranking_scores = collect_reranking(reranking_model, dataloader_test)

    reranking_end_time = time.time()


    query_scores = {}
    reranking_doc_ids = reranking_docs_tokens_df['doc_id'].tolist()

    for i, match_score in enumerate(reranking_scores):
        which_query = reranking_doc_ids[i]

        if which_query not in query_scores:
            query_scores[which_query] = []

        query_scores[which_query].append(match_score)

    # Consolidate the scores according to the defined approach.
    
    consolidated_scores = []

    for document, scores in query_scores.items():
        if consolidation_approach == 'mean':
            consolidated_scores.append({'doc_id': document, 'score': np.mean(scores)})
        else:
            consolidated_scores.append({'doc_id': document, 'score': np.max(scores)})
            
    consolidated_scores_df = pd.DataFrame(consolidated_scores).sort_values('score', ascending=False)  


    handling_overflow_end_time = time.time()

    if verbose:
        print("Reranking DL time: {}\nReranking overflow handling time: {}\nQuery execution time: {}".format(handling_overflow_end_time - reranking_start_time,
                                                                                                            handling_overflow_end_time - reranking_end_time,
                                                                                                            handling_overflow_end_time - query_start_time))

    return consolidated_scores_df, hits_df, handling_overflow_end_time - query_start_time

In [54]:
def check_trec_covid_result(result_df, query_id, consolidation_approach='mean', comment='BM25'):

    TREC_RESULT_LINE_FORMAT="{}\tQ0\t{}\t{}\t{}\t{}\n"
    TREC_COVID_QRELS_FORMAT="trec_covid_qrels_{:02}.tsv"

    test_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    reranked_run = PYSERINI_TEST_RUN_RERANKED_FILENAME_FORMAT.format(test_timestamp, consolidation_approach)

    if not os.path.exists(TREC_COVID_RERANKING_RUNS_FOLDER):
        os.makedirs(TREC_COVID_RERANKING_RUNS_FOLDER)

    with open(os.path.join(TREC_COVID_RERANKING_RUNS_FOLDER, reranked_run), 'w') as outputFile:
        for i, row in result_df.iterrows():
            outputFile.write(TREC_RESULT_LINE_FORMAT.format(query_id, row['doc_id'], i + 1, row['score'], comment))
    
    query_qrels = TREC_COVID_QRELS_FORMAT.format(query_id)

    result = !{TREC_EVAL_FULLPATH} -c -mrecall.1000 -mmap -mndcg_cut.10 -mrecip_rank.100 \
                    {WORKING_FOLDER}/{query_qrels} {WORKING_FOLDER}/{TREC_COVID_RERANKING_RUNS_FOLDER}/{reranked_run}

    results = {}

    for line in result:

        line = line.split('\t')

        results[line[0].strip()] = np.float32(line[-1])

    return results, reranked_run

### Loop through TREC COVID queries to execute them

In [57]:
def trec_covid_queries(k=1000, k2=1000, batch_size=batch_size, verbose=False):

    queries_df = pd.read_csv(TREC_COVID_QUERIES, sep='\t', header=None, names=['query_id', 'query_text'])

    queries_results = []    

    for i, row in queries_df.iterrows():
        if verbose:
            if i > 0:
                print("\n")

            print("Query {}: {}".format(i, row['query_text']))

        reranking_scores_df, BM25_hits_df, query_duration = execute_query(row['query_text'], 
                                                                          k=k,
                                                                          k2=k2, 
                                                                          batch_size=batch_size)

        BM25_results = check_trec_covid_result(BM25_hits_df, query_id=row['query_id'])
        pipeline_results = check_trec_covid_result(reranking_scores_df, query_id=row['query_id'])

        queries_results.append({'query_id': row['query_id'],
                                'ndcg@10 BM25': BM25_results[0]['ndcg_cut_10'],
                                'ndcg@10 pipeline': pipeline_results[0]['ndcg_cut_10'],
                                'duration': query_duration})
    
    query_results_df = pd.DataFrame(queries_results)

    query_results_df.loc['mean'] = query_results_df.mean()

    return query_results_df

In [58]:
!nvidia-smi

Thu May 11 02:03:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    53W / 400W |  17525MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [63]:
k_1000_results_df = trec_covid_queries(batch_size=300)

In [61]:
k_100_results_df = trec_covid_queries(k=1000, k2=100, batch_size=512)

In [67]:
k_500_results_df = trec_covid_queries(k=500, k2=500, batch_size=512)

In [69]:
k_300_results_df = trec_covid_queries(k=300, k2=300, batch_size=300)

In [71]:
k_2000_k2_500_results_df = trec_covid_queries(k=2000, k2=500, batch_size=512)

In [ ]:
with open(RESULTS_FILENAME, "wb") as output_file:
    pickle.dump({"k_100_results_df": k_100_results_df,
                 "k_300_results_df": k_300_results_df,
                 "k_500_results_df": k_500_results_df,
                 "k_1000_results_df": k_1000_results_df}, output_file, pickle.HIGHEST_PROTOCOL)

In [62]:
k_100_results_df.tail()

,query_id,ndcg@10 BM25,ndcg@10 pipeline,duration
46,47.0,0.260100,0.662000,0.397461
47,48.0,0.450900,0.963300,0.390841
48,49.0,0.332700,0.554700,0.438766
49,50.0,0.000000,0.587800,0.390975
mean,25.5,0.417724,0.751644,0.427348


In [70]:
k_300_results_df.tail()

,query_id,ndcg@10 BM25,ndcg@10 pipeline,duration
46,47.0,0.000000,0.532600,0.703598
47,48.0,0.496400,0.932200,0.707795
48,49.0,0.401000,0.657900,0.755143
49,50.0,0.000000,0.612800,0.701488
mean,25.5,0.279656,0.759322,0.714878


In [68]:
k_500_results_df.tail()

,query_id,ndcg@10 BM25,ndcg@10 pipeline,duration
46,47.0,0.078400,0.769400,1.070935
47,48.0,0.119900,0.935100,1.057332
48,49.0,0.151400,0.794900,1.144562
49,50.0,0.031800,0.612800,1.086617
mean,25.5,0.227222,0.756814,1.091818


In [64]:
k_1000_results_df.tail()

,query_id,ndcg@10 BM25,ndcg@10 pipeline,duration
46,47.0,0.000000,0.847200,2.052452
47,48.0,0.218700,0.935100,1.981107
48,49.0,0.140200,0.815100,2.126333
49,50.0,0.000000,0.612800,1.965848
mean,25.5,0.170632,0.756096,2.049879


In [72]:
k_2000_k2_500_results_df.tail()

,query_id,ndcg@10 BM25,ndcg@10 pipeline,duration
46,47.0,0.078400,0.769400,1.185823
47,48.0,0.119900,0.935100,1.175784
48,49.0,0.151400,0.794900,1.274060
49,50.0,0.031800,0.612800,1.188015
mean,25.5,0.227222,0.756814,1.218986
